In [ ]:
from keras.models import load_model
import tensorflow as tf
from PIL import Image
import numpy as np
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

categories = ["버스","화물차","해치백","승용차","SUV","트럭","승합차"]

#load models
Xception = tf.keras.models.load_model('./VTIP_model_Xception.h5') 
Resnet = tf.keras.models.load_model('./VTIP_model_Resnet50.h5') 
Densnet = tf.keras.models.load_model('./VTIP_model_Densenet121.h5') 
Mobilenet = tf.keras.models.load_model('./VTIP_model_mobilenet.h5') 


In [ ]:
data_dir = "./DataSet"

batch_size = 32
img_height = 128
img_width = 128

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take((2*val_batches) // 3)
val_ds = val_ds.skip((2*val_batches) // 3)

In [ ]:
models = [Xception, Resnet, Densnet, Mobilenet]
models_name = ['Xception', 'Resnet', 'Densnet', 'Mobilenet']
#CNN은 편차가 커 삭제

for i, model in enumerate (models):
    model._name = models_name[i]
    print(model._name)


In [ ]:
model_input = tf.keras.Input(shape=(128, 128, 3))

model_outputs = [model(model_input) for model in models]


out = tf.keras.layers.Add()(model_outputs)

ensemble_model = tf.keras.Model(inputs=model_input, outputs=out)

ensemble_model._name = 'VITP'

ensemble_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

ensemble_model.summary()


In [ ]:
loss, accuracy = ensemble_model.evaluate(test_ds)
print('Test accuracy :', accuracy)
print('Test loss :', loss)


In [ ]:
# (멀티) 분류 문제인 경우 softmax 적용 후 argmax로 계산
# make predictions
img_path = "./"+"bus.jpg"
img = Image.open(img_path)
img = img.convert("RGB")
img = img.resize((128, 128))
data = np.asarray(img)
data = np.expand_dims(data, axis=0)

X_preds = Xception.predict(data)
R_preds = Resnet.predict(data)
D_preds = Densnet.predict(data)
C_preds = CNN.predict(data)
M_preds = Mobilenet.predict(data)
E_preds = ensemble_model.predict(data)

print(' ')

def pred(x, name):
    print(name)
    print(x[0])
    print(' ')
    index = x.argmax()
        #print(index)
    return categories[index]



X = pred(X_preds, 'Xception')
R = pred(R_preds, 'Resnet50')
D = pred(D_preds, 'Densenet121')
M = pred(M_preds, 'MobileNetV2')
E = pred(E_preds, 'ADD_Ensemble')



In [ ]:
from keras.models import load_model
ensemble_model.save('VTIP_Ensemble.h5')
ensemble_model.save_weights('VTIP_Ensemble_weights.h5')
